# Underwater Sound Analysis
## General imports

In [56]:
import os
import csv
import librosa
import soundfile as sf
from collections import Counter

## Audio Features

In [68]:
def scomponi_percorso_audio(percorso):
    # Scomponi il percorso in elementi separati
    elementi = percorso.split('/')
    
    # Estrai il nome del dataset, il tipo di target e il nome del file audio
    dataset = elementi[0]
    tipo_target = elementi[1]
    nome_target = elementi[2]
    nome_file_audio = elementi[-1]
    
    return dataset, tipo_target, nome_target, nome_file_audio

def extract_audio_features(audio_path):
    
    y, sr = librosa.load(audio_path, sr=None, mono=False)
    duration = librosa.get_duration(y=y, sr=sr)
    channels = 'Mono' if y.ndim == 1 else 'Stereo'
    return {'Audio': audio_path, 'Durata (s)': duration, 'Canali': channels, 'Frequenza(Hz)': sr}


In [71]:


# Lista per memorizzare i percorsi dei file audio
audio_files = []

# Scorrere ricorsivamente le cartelle nel dataset
for root, dirs, files in os.walk('Dataset'):
    for file in files:
        if file.endswith(('.wav','.mp3')):
           audio_files.append(os.path.join(root, file))
audio_features = []

for file in audio_files:

    feature1 = extract_audio_features(file)
    print(file)
    audio_features.append(feature1)

Dataset\Non-Target\Ambient Noise\0f9867d8-fcf7-40d1-b846-3f7fb8100eb7.mp3
Dataset\Non-Target\Ambient Noise\2b68b3a9-7c0d-4452-bbe5-cc99b5d4fb61.mp3
Dataset\Non-Target\Ambient Noise\2f574049-3325-446c-a8ac-2e5b31e9e32f.mp3
Dataset\Non-Target\Ambient Noise\30679527-fd7a-4cad-8c7d-cb1bcc07604d.mp3
Dataset\Non-Target\Ambient Noise\f9652ad2-39f5-41b4-a8e4-89171a864f03.mp3
Dataset\Non-Target\Antarctic Minke Whale\Bioduck.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-1.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-2.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-3.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-4.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-5.wav
Dataset\Non-Target\Arnoux’s Beaked Whale\Arnoux-beaked-whales-rogers-6.wav
Dataset\Non-Target\Atlantic cod\SanctSound_SB01_01_atlanticcod_20181113T024154Z.wav
Dataset\Non-Target\Atlantic croaker\Croaker

In [72]:

# Esempio di utilizzo
percorso_audio = "Dataset\Target\Tanker\20170723-238_052014.wav"
dataset, tipo_target, nome_target, nome_file_audio = scomponi_percorso_audio(percorso_audio)

print("Dataset:", dataset)
print("Tipo di Target:", tipo_target)
print("Nome del Target:", nome_target)
print("Nome del file audio:", nome_file_audio)

IndexError: list index out of range

## Printing equals audio files into Dataset

In [60]:
###########
file_names = [os.path.basename(file) for file in audio_files]

file_counts = Counter(file_names)
duplicates_info = {}
# Creare un dizionario per mantenere traccia del percorso del file originale
original_paths = {}

for file_path in audio_files:
    file_name = os.path.basename(file_path)
    if file_counts[file_name] > 1:
        if file_name not in duplicates_info:
            duplicates_info[file_name] = {
                "count": file_counts[file_name],
                "paths": [file_path]
            }
            # Memorizza il percorso del file originale
            original_paths[file_name] = file_path
        else:
            duplicates_info[file_name]["count"] += 1
            duplicates_info[file_name]["paths"].append(file_path)

# Stampare informazioni sui file duplicati
total_duplicate_count = sum(info["count"] for info in duplicates_info.values())
print(f"Numero totale di file audio duplicati: {total_duplicate_count/3}")

for file_name, info in duplicates_info.items():
    print(f"Nome del file duplicato: {file_name}")
    print(f"Percorsi duplicati ({info['count']-1}):")
    for path in info["paths"]:
        print(path)
    print(f"Percorso del file originale: {original_paths[file_name]}")
    print()

Numero totale di file audio duplicati: 25.0
Nome del file duplicato: 72021005.wav
Percorsi duplicati (2):
Dataset\Non-Target\Bearded Seal\72021005.wav
Dataset\Non-Target\Bowhead Whale\72021005.wav
Percorso del file originale: Dataset\Non-Target\Bearded Seal\72021005.wav

Nome del file duplicato: 7202100T.wav
Percorsi duplicati (2):
Dataset\Non-Target\Bearded Seal\7202100T.wav
Dataset\Non-Target\Bowhead Whale\7202100T.wav
Percorso del file originale: Dataset\Non-Target\Bearded Seal\7202100T.wav

Nome del file duplicato: 7202100V.wav
Percorsi duplicati (2):
Dataset\Non-Target\Bearded Seal\7202100V.wav
Dataset\Non-Target\Bowhead Whale\7202100V.wav
Percorso del file originale: Dataset\Non-Target\Bearded Seal\7202100V.wav

Nome del file duplicato: 7202100Z.wav
Percorsi duplicati (2):
Dataset\Non-Target\Bearded Seal\7202100Z.wav
Dataset\Non-Target\Bowhead Whale\7202100Z.wav
Percorso del file originale: Dataset\Non-Target\Bearded Seal\7202100Z.wav

Nome del file duplicato: 78018002.wav
Percor

## Manage Features in csv file

In [61]:
csv_file = 'Features.csv'
def scrivi_csv(audio_features, csv_file):
    if not audio_features:
        return  # Se la lista è vuota, non fare nulla

    # Ottieni le chiavi dal primo dizionario per usarle come intestazioni
    keys = audio_features[0].keys()

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        
        # Scrivi l'intestazione
        #writer.writeheader()
        
        # Scrivi le caratteristiche
        for features in audio_features:
            writer.writerow(features)

scrivi_csv(audio_features, csv_file)

## Calculate the class distribution

In [62]:
import pandas as pd
import matplotlib.pyplot as plt

# Leggi il file CSV con una codifica diversa
df = pd.read_csv('Features.csv', encoding='cp1252')
pd.set_option('display.max_rows', None)
# Mostra il dataframe
display(df)

# Esempio di grafico a barre delle classi nel dataset
plt.figure(figsize=(18, 10))
df['Label'].value_counts().plot(kind='bar')
plt.title('Distribuzione delle classi')
plt.xlabel('Classe')
plt.ylabel('Numero di campioni')
plt.show()

,Dataset\Non-Target\Ambient Noise\0f9867d8-fcf7-40d1-b846-3f7fb8100eb7.mp3,31.207619047619048,Mono,44100
0,Dataset\Non-Target\Ambient Noise\2b68b3a9-7c0d...,31.207619,Mono,44100
1,Dataset\Non-Target\Ambient Noise\2f574049-3325...,31.207619,Mono,44100
2,Dataset\Non-Target\Ambient Noise\30679527-fd7a...,31.207619,Mono,44100
3,Dataset\Non-Target\Ambient Noise\f9652ad2-39f5...,31.207619,Mono,44100
4,Dataset\Non-Target\Antarctic Minke Whale\Biodu...,20.000000,Mono,1000
5,Dataset\Non-Target\Arnoux’s Beaked Whale\Arnou...,9.992971,Mono,44100
6,Dataset\Non-Target\Arnoux’s Beaked Whale\Arnou...,8.668526,Mono,44100
7,Dataset\Non-Target\Arnoux’s Beaked Whale\Arnou...,10.292971,Mono,44100
8,Dataset\Non-Target\Arnoux’s Beaked Whale\Arnou...,9.993016,Mono,44100
9,Dataset\Non-Target\Arnoux’s Beaked Whale\Arnou...,19.993016,Mono,44100


KeyError: 'Label'

<Figure size 1800x1000 with 0 Axes>

## Audio Duration Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="whitegrid")

# Lista delle durate
durations = [feat['Durata (s)'] for feat in audio_features]

plt.figure(figsize=(12, 8))
plt.hist(durations, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribuzione della Durata degli Audio', fontsize=16, weight='bold')
plt.xlabel('Durata (s)', fontsize=14)
plt.ylabel('Numero di Campioni', fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Aggiunta annotazioni
median_duration = np.median(durations)
plt.axvline(median_duration, color='red', linestyle='dashed', linewidth=1)
plt.text(median_duration, plt.ylim()[1] * 0.9, f'Median: {median_duration:.2f}s', color='red', fontsize=12)

# Aggiungi griglia
plt.grid(True, linestyle='--', alpha=0.6)

# Visualizza il grafico
plt.show()


## Audio frequencies distribution
### It is used to understand at what frequency to change

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tabulate import tabulate

# Imposta il tema di Seaborn per migliorare l'estetica
sns.set_theme(style="whitegrid")

# Frequenze di campionamento (assumendo che audio_features sia definito)
frequencies = [feat['Frequenza di campionamento (Hz)'] for feat in audio_features]

# Creazione del grafico della distribuzione della frequenza di campionamento
plt.figure(figsize=(12, 8))  # Dimensioni maggiori per una migliore leggibilità
plt.hist(frequencies, bins=20, color='salmon', edgecolor='black', alpha=0.7)
plt.title('Distribuzione della Frequenza di Campionamento', fontsize=16, weight='bold')
plt.xlabel('Frequenza di Campionamento (Hz)', fontsize=14)
plt.ylabel('Numero di Campioni', fontsize=14)

# Migliora le etichette degli assi
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Aggiungi annotazioni per la frequenza massima e la mediana
freq_max = max(frequencies)
median_freq = np.median(frequencies)
plt.axvline(freq_max, color='blue', linestyle='dashed', linewidth=1.5)
plt.axvline(median_freq, color='green', linestyle='dashed', linewidth=1.5)
plt.text(freq_max, plt.ylim()[1] * 0.9, f'Max: {freq_max} Hz', color='blue', fontsize=12, ha='right')
plt.text(median_freq, plt.ylim()[1] * 0.7, f'Median: {median_freq} Hz', color='green', fontsize=12, ha='right')

plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

hist, bins = np.histogram(frequencies, bins=20)





### La mediana è il valore centrale che separa la metà superiore dei dati dalla metà inferiore. Essa non è influenzata dai valori anomali, come la media, e fornisce una misura più rappresentativa della frequenza di campionamento tipica.

In [ ]:
tabella_dati = []
for i in range(len(hist)):
    tabella_dati.append((f"{bins[i]:.2f} - {bins[i+1]:.2f} Hz", hist[i]))
# Stampa della tabella
print(tabulate(tabella_dati, headers=["Intervallo di frequenza", "Numero di campioni"], tablefmt="pretty"))

In [ ]:
statistiche_frequenze = [
    ("Frequenza massima", freq_max),
    ("Mediana delle frequenze", median_freq)
]

print(tabulate(statistiche_frequenze, headers=["Statistiche", "Valore (Hz)"], tablefmt="pretty"))

## It is used to see the paths of the audios that we are going to cut

In [ ]:
def estrai_path_e_conta_frequenze(audio_features, frequenza_minima):
    # Filtra i file audio per la frequenza di campionamento minima specificata
    paths = [feat['Audio'] for feat in audio_features if feat['Frequenza di campionamento (Hz)'] >= frequenza_minima]
    total_frequencies = len(paths)
    return paths, total_frequencies

# Frequenza minima desiderata (Hz)
frequenza_minima = 200001

# Estrazione dei path e conteggio delle frequenze per la frequenza minima specificata
paths, total_frequencies = estrai_path_e_conta_frequenze(audio_features, frequenza_minima)

# Stampa dei percorsi estratti
print(f"Percorsi dei file audio con frequenza di campionamento ≥ {frequenza_minima} Hz:")
for path in paths:
    print(path)

# Stampa del numero totale di frequenze trovate
print(f"\nNumero totale di frequenze trovate: {total_frequencies}")

## Mono/Stereo counting

In [ ]:
channels = [feat['Canali'] for feat in audio_features]

# Grafico del conteggio di mono/stereo
plt.figure(figsize=(10, 6))
plt.bar(['Mono', 'Stereo'], [channels.count('Mono'), channels.count('Stereo')], color=['lightgreen', 'lightcoral'], edgecolor=['black','black'])
plt.title('Conteggio di Mono/Stereo')
plt.xlabel('Tipo di Canale')
plt.ylabel('Conteggio')
plt.grid(axis='y')
plt.show()

## Depth Bit Counts

In [ ]:
for root, dirs, files in os.walk('Dataset'):
    for file in files:
        if file.endswith(('.wav')):
           audio_files.append(os.path.join(root, file))
# Creare un dizionario per tenere traccia del conteggio di ciascun valore
depth_bit_counts = {}

for file in audio_files:
    audio = sf.SoundFile(file)
    depth_bit = audio.subtype
    
    # Aggiungere il valore al dizionario o incrementare il conteggio se già presente
    if depth_bit in depth_bit_counts:
        depth_bit_counts[depth_bit] += 1
    else:
        depth_bit_counts[depth_bit] = 1

# Estrai i dati per il grafico
depth_bits = list(depth_bit_counts.keys())
counts = list(depth_bit_counts.values())

# Creare il grafico a barre con dimensioni specificate
plt.figure(figsize=(10, 6))  # Imposta la dimensione della figura a 12x8 pollici
plt.bar(depth_bits, counts, color='lightgreen', edgecolor='black')
plt.xlabel('Depth Bit', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Depth Bit Counts', fontsize=16, weight='bold')

# Migliora le etichette degli assi
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Aggiungi una griglia per migliorare la leggibilità
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostra il grafico
plt.show()